# SQL. Работа с базой данных, хранящей информацию о венчурных фондах и инвестициях в компании-стартапы

**Задача проекта** - проанализировать данные о фондах и инвестициях и написать запросы к базе. 

## Задание 1

Посчитайте, сколько компаний закрылось.

In [ ]:
'''
SELECT COUNT(status)
FROM company
WHERE status = 'closed';
'''

## Задание 2

Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле `funding_total`.

In [ ]:
'''
SELECT funding_total
FROM company
WHERE category_code = 'news'
AND country_code = 'USA'
ORDER BY funding_total DESC;
'''

## Задание 3

Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
'''
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash' 
AND EXTRACT(YEAR FROM CAST(acquired_at AS DATE)) BETWEEN 2011 AND 2013;
'''

## Задание 4

Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на `Silver`.

In [ ]:
'''
SELECT first_name,
    last_name,
    twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';
'''

## Задание 5

Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку `money`, а фамилия начинается на `K`.

In [ ]:
'''
SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
AND last_name LIKE 'K%';
'''

## Задание 6

Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
'''
SELECT country_code,
     SUM(funding_total)
FROM company
GROUP BY  country_code
ORDER BY SUM(funding_total) DESC;
'''

## Задание 7

Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.

Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
'''
SELECT funded_at,
    MIN(raised_amount),
    MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) <> 0 
AND MIN(raised_amount) <> MAX(raised_amount);
'''

## Задание 8

Создайте поле с категориями:
* Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию `high_activity`.
* Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию `middle_activity`.
* Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию `low_activity`.

Отобразите все поля таблицы `fund` и новое поле с категориями.

In [ ]:
'''
SELECT *,
      CASE
          WHEN invested_companies >= 100 THEN 'high_activity'
          WHEN invested_companies BETWEEN 20 AND 100 THEN 'middle_activity'
          WHEN invested_companies < 20 THEN 'low_activity'
      END
FROM fund;
'''

## Задание 9

Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
'''
SELECT ROUND(AVG(investment_rounds)),
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity
FROM fund
GROUP BY activity
ORDER BY ROUND(AVG(investment_rounds));
'''

## Задание 10

Выгрузите таблицу с десятью самыми активными инвестирующими странами. Активность страны определите по среднему количеству компаний, в которые инвестируют фонды этой страны.

Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды, основанные с 2010 по 2012 год включительно.

Исключите из таблицы страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. Отсортируйте таблицу по среднему количеству компаний от большего к меньшему, а затем по коду страны в лексикографическом порядке.

Для фильтрации диапазона по годам используйте оператор `BETWEEN`.

In [ ]:
'''
SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
FROM fund
WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies) <> 0
ORDER BY AVG(invested_companies) DESC
LIMIT 10;
'''

## Задание 11

Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
'''
SELECT first_name,
       last_name,
       instituition
FROM people 
LEFT OUTER JOIN education ON people.id = education.person_id;
'''

## Задание 12

Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
'''
SELECT c.name,
       COUNT(DISTINCT instituition)
FROM company AS c
INNER JOIN people AS p ON c.id = p.company_id
INNER JOIN education AS e ON p.id = e.person_id
GROUP BY c.name
ORDER BY COUNT(DISTINCT instituition) DESC
LIMIT 5;
'''

## Задание 13

Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
'''
SELECT DISTINCT name
FROM company AS c
INNER JOIN funding_round AS f ON c.id = f.company_id
WHERE status = 'closed'
AND is_first_round = 1
AND is_last_round = 1;
'''

## Задание 14

Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
'''
SELECT DISTINCT p.id
FROM people AS p
INNER join company AS c ON p.company_id = c.id
INNER JOIN funding_round AS f ON c.id = f.company_id
WHERE status = 'closed'
AND is_first_round = 1
AND is_last_round = 1;
'''

## Задание 15

Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
'''
SELECT DISTINCT p.id,
                e.instituition
FROM people AS p JOIN education AS e ON p.id = e.person_id
WHERE company_id IN (SELECT id
                     FROM company
                     WHERE id IN (SELECT company_id
                                  FROM funding_round
                                  WHERE is_first_round = 1 AND is_last_round = 1)
AND status = 'closed');
'''

## Задание 16

Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
'''
COUNT(e.instituition) AS total_instituition
FROM people AS p JOIN education AS e ON p.id = e.person_id
WHERE company_id in (SELECT id
                     FROM company 
                     WHERE id in (SELECT company_id
                                  FROM funding_round
                                  WHERE is_first_round = 1 AND is_last_round = 1)
                     AND status = 'closed')
GROUP BY p.id;
'''

## Задание 17 

Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
'''
SELECT AVG(ins)
FROM (SELECT COUNT(e.instituition) AS ins
      FROM people AS p JOIN education AS e ON p.id = e.person_id
WHERE company_id in (SELECT id
                     FROM company 
                     WHERE id in (SELECT company_id
                                  FROM funding_round
                                  WHERE is_first_round = 1 AND is_last_round = 1)
                     AND status = 'closed')
GROUP BY p.id) nested;
'''

## Задание 18

Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.

*(сервис, запрещённый на территории РФ)

In [ ]:
'''
SELECT AVG(ins)
FROM (SELECT COUNT(e.instituition) AS ins
      FROM people AS p JOIN education AS e ON p.id = e.person_id
      WHERE company_id in (SELECT id
                          FROM company 
                          WHERE name = 'Facebook')
      GROUP BY p.id) nested;
'''

## Задание 19

Составьте таблицу из полей:
* name_of_fund — название фонда;
* name_of_company — название компании;
* amount — сумма инвестиций, которую привлекла компания в раунде.

В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
'''
SELECT f.name AS name_of_fund,
       c.name AS name_of_company,
       fr.raised_amount AS amount
FROM investment AS i
INNER JOIN company AS c ON i.company_id = c.id
INNER JOIN fund AS f ON i.fund_id = f.id
INNER JOIN funding_round AS fr ON i.funding_round_id = fr.id
WHERE c.milestones > 6
AND EXTRACT(YEAR FROM CAST(funded_at AS DATE)) BETWEEN 2012 AND 2013;
'''

## Задание 20

Выгрузите таблицу, в которой будут такие поля:
* название компании-покупателя;
* сумма сделки;
* название компании, которую купили;
* сумма инвестиций, вложенных в купленную компанию;
* доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.

Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
'''
SELECT c.name,
       a.price_amount,
       co.name,
       co.funding_total,
       ROUND(a.price_amount/co.funding_total)
FROM acquisition AS a
INNER JOIN company AS c ON a.acquiring_company_id = c.id
INNER JOIN company AS co ON a.acquired_company_id = co.id
WHERE a.price_amount!=0 AND co.funding_total!=0
ORDER BY a.price_amount DESC, co.name ASC
LIMIT 10;
'''

## Задание 21 

Выгрузите таблицу, в которую войдут названия компаний из категории `social`, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
'''
SELECT c.name,
       EXTRACT(MONTH from CAST(funded_at AS DATE)) 
FROM company AS c
INNER JOIN funding_round AS fr ON c.id = fr.company_id
WHERE category_code = 'social'
AND EXTRACT(YEAR from CAST(funded_at AS DATE)) BETWEEN 2010 AND 2013;
'''

## Задание 22 

Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
* номер месяца, в котором проходили раунды;
* количество уникальных названий фондов из США, которые инвестировали в этом месяце;
* количество компаний, купленных за этот месяц;
* общая сумма сделок по покупкам в этом месяце.

In [ ]:
'''
WITH
fu AS (SELECT COUNT(DISTINCT fd.id) AS funds,
              EXTRACT(MONTH FROM CAST(fr.funded_at AS timestamp)) AS mo
              FROM fund AS fd
              LEFT JOIN investment AS i ON fd.id=i.fund_id
              LEFT JOIN funding_round AS fr ON i.funding_round_id = fr.id
              LEFT JOIN company AS c ON c.id=fr.company_id
              WHERE fd.country_code = 'USA' 
              AND EXTRACT(YEAR FROM CAST(fr.funded_at AS timestamp)) BETWEEN '2010' AND '2013' 
              GROUP BY EXTRACT(MONTH FROM CAST(fr.funded_at AS timestamp))),
ac AS (SELECT COUNT(acquired_company_id) AS cc,
              SUM(price_amount) AS total,
              EXTRACT(MONTH FROM CAST(acquired_at AS timestamp)) AS mo
              FROM acquisition
              WHERE EXTRACT(YEAR FROM CAST(acquired_at AS timestamp)) BETWEEN '2010' AND '2013'
              GROUP BY EXTRACT(MONTH FROM CAST(acquired_at AS timestamp)))   
SELECT fu.mo AS number_of_month,
       fu.funds,
       ac.cc,
       ac.total
FROM fu
JOIN ac ON fu.mo=ac.mo;
'''

## Задание 23

Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
'''
WITH
i11 as(SELECT country_code,
            AVG(funding_total) AS i1
            FROM company
            WHERE EXTRACT(YEAR FROM founded_at) = 2011
            GROUP BY country_code),
i12 as(SELECT country_code,
            AVG(funding_total) AS i2
            FROM company
            WHERE EXTRACT(YEAR FROM founded_at) = 2012
            GROUP BY country_code),           
i13 as(SELECT country_code,
            AVG(funding_total) AS i3
            FROM company
            WHERE EXTRACT(YEAR FROM founded_at) = 2013
            GROUP BY country_code)
SELECT
i11.country_code,
i1,
i2,
i3
FROM i11
INNER JOIN i12 ON i12.country_code=i11.country_code
INNER JOIN i13 ON i11.country_code=i13.country_code
ORDER BY i1 DESC;
'''